Hier könnte Ihre Werbung stehen.

In [1]:
import subprocess
import numpy as np
import sfs

In [2]:

if [False,True][0]:


    execNEST_ndarray = sfs.execNEST(
        spectrum_dict = {
            "numEvts" : [10, 10, 10, 10],
            "type_interaction" : "ER",
            "E_min[keV]" : [100, 100, 100, 100],
            "E_max[keV]" : [100, 100, 100, 100],
            "field_drift[V/cm]" : "200",
        },
        detector_dict = {
            "g1" : 1.9,
            "g1_gas" : 1.9,
        },
        flag_verbose = True,
    )


    print(len(execNEST_ndarray))


#### write DARWIN baseline header file

In [3]:

if [False,True][0]:

    sfs.convert_detector_dict_into_detector_header(
        detector_dict = sfs.darwin_baseline_detector_dict,
        detector_name = "darwin_testdetector.hh",
        abspath_output_list = [sfs.abspath_sfs_repo_detectors],
        flag_verbose = True,
    )



convert_detector_dict_into_detector_header: initializing
	detector_name: darwin_testdetector
convert_detector_dict_into_detector_header: adding initial lines to 'line_list'.
convert_detector_dict_into_detector_header: adding 'detector_dict' parameters to 'line_list'.
convert_detector_dict_into_detector_header: adding final lines to 'line_list'.
convert_detector_dict_into_detector_header: writing 'line_list' into header files.
	saved: /home/daniel/Desktop/arbeitsstuff/sfs/github_repo_v2/detectors/darwin_testdetector.hh


#### install DARWIN baseline Detector

In [6]:

if [False,True][0]:

    sfs.install_detector_header_file(
        abspathfile_new_detector_hh = sfs.abspath_sfs_repo_detectors +"darwin_testdetector.hh",
        flag_clean_reinstall = True,
        flag_verbose = True,
    )



install_detector_header_file: initialization
install_detector_header_file: copying '/home/daniel/Desktop/arbeitsstuff/sfs/github_repo_v2/detectors/darwin_testdetector.hh' into /home/daniel/Desktop/arbeitsstuff/sfs/nest_v_2_3_9/nest/include/Detectors/
stdout: ''
stderr: ''
install_detector_header_file: reading '/home/daniel/Desktop/arbeitsstuff/sfs/nest_v_2_3_9/nest/src/execNEST.cpp'
install_detector_header_file: modifying '/home/daniel/Desktop/arbeitsstuff/sfs/nest_v_2_3_9/nest/src/execNEST.cpp'
	inserted '#include "darwin_testdetector.hh"
'
	inserted '  auto* detector = new darwin_testdetector();
'
	inserted '    cerr << "You are currently using the darwin_testdetector detector." << endl
'
install_detector_header_file: saving modified version of '/home/daniel/Desktop/arbeitsstuff/sfs/nest_v_2_3_9/nest/src/execNEST.cpp'
install_detector_header_file: performing clean reinstall

make_clean_reinstall: initialization
make_clean_reinstall: defining shell commands in 'cmd_list'
make_clean_r